# Data Imports/Pre-Processing:

In [ ]:
# Importing UNSW-NB15 data.
!pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/uc?id=1vSt-P_lya1Rzlyddivvr9byVei-goIZ0

# Converting the csv file to dataframe.
import pandas as pd
df_UNSW = pd.read_csv('UNSW_all.csv')

In [ ]:
# Creating testing/training data for UNSW-NB15.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
y = df_UNSW.label
x = df_UNSW.drop(labels = ['label'], axis = 1)
x = scaler.fit_transform(x)
X_train_UNSW, X_test_UNSW, y_train_UNSW, y_test_UNSW = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
# Importing Kyoto-2006 data.
!pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/uc?id=1z479qXYwfwg7sqFhGkGrPpsxd8oHhiNF

# Converting the csv file to a dataframe.
df_KYOTO = pd.read_csv("Kyoto_all.csv")

In [ ]:
# Creating the testing/training data for KYOTO.
y = df_KYOTO.Label
x = df_KYOTO.drop(labels = ['Label'], axis = 1)
scaler = StandardScaler()
x = scaler.fit_transform(x)
X_train_KYOTO, X_test_KYOTO, y_train_KYOTO, y_test_KOYTO = train_test_split(x, y, test_size=0.33, random_state=42)

# Convolutional Neural Network:

In [ ]:
import tensorflow as tf

# Define the input shape.
input_shape = (106,1)

# Create a CNN with the different layers (Conv1D, MaxPooling, Dropout, and Dense).
cnn = tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
cnn.add(tf.keras.layers.MaxPooling1D())
cnn.add(tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling1D())
cnn.add(tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling1D())
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.2))
cnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy loss and Adam optimization.
cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Get the initial weights of the architecture. 
initial_weights = cnn.get_weights()

In [ ]:
# Train the model on the UNSW-NB15 data.
cnn.fit(X_train_UNSW, y_train_UNSW, batch_size=64, epochs=10)

# Get the accuracy of the model on the test data.
loss, accuracy = cnn.evaluate(X_test_UNSW, y_test_UNSW)
print('Test accuracy:', accuracy)

In [ ]:
# Unfit the model.
cnn.set_weights(initial_weights)

# Train the model on the KYOTO-2006 data.
cnn.fit(X_train_KYOTO, y_train_KYOTO, batch_size=64, epochs=10)

# Get the accuracy of the model on the test data.
loss, accuracy = cnn.evaluate(X_test_KYOTO, y_test_KOYTO)
print('Test accuracy:', accuracy)

# Multi-Layer Perceptron:

In [ ]:
import tensorflow as tf

# Define the input shape.
input_shape = (106,)

# Create the MLP Model using the Dense layers.
mlp = tf.keras.Sequential()
mlp.add(tf.keras.layers.Dense(212, activation='relu', input_shape=input_shape))
mlp.add(tf.keras.layers.Dense(100, activation='relu'))
mlp.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy loss and Adam optimization.
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Get the initial weights of the architecture. 
initial_weights = mlp.get_weights()

In [ ]:
# Train the model on the UNSW-NB15 data.
mlp.fit(X_train_UNSW, y_train_UNSW, batch_size=64, epochs=10)

# Get the accuracy of the model on the test data.
loss, accuracy = mlp.evaluate(X_test_UNSW, y_test_UNSW)
print('Test accuracy:', accuracy)

In [ ]:
# Unfit the model.
mlp.set_weights(initial_weights)

# Train the model on the KYOTO-2006 data.
mlp.fit(X_train_KYOTO, y_train_KYOTO, batch_size=64, epochs=10)

# Get the accuracy of the model on the test data.
loss, accuracy = mlp.evaluate(X_test_KYOTO, y_test_KOYTO)
print('Test accuracy:', accuracy)

# Variational Autoencoder:

In [ ]:
# Creating the dimensions for VAE.
import keras
from keras import layers
original_dim = 106
intermediate_dim = 20
latent_dim = 2

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

In [ ]:
# Method to sample new points from latent space.
from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

In [ ]:
# Create encoder.
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='sigmoid')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# Instantiate VAE model.
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

In [ ]:
# Train the model using the end-to-end model, with a custom loss function (reconstruction term + KL divergence regularization term)
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam', metrics = ['accuracy'])

# Get the initial weights of the architecture. 
initial_weights = vae.get_weights()

In [ ]:
# Train the VAE on UNSW-NB15.
vae.fit(X_train_UNSW, y_train_UNSW,
        epochs=50,
        batch_size=2250,
        validation_data=(X_test_UNSW, y_test_UNSW))

In [ ]:
# Print the accuracy on the test data.
loss, accuracy = vae.evaluate(X_test_UNSW, y_test_UNSW)
print('Test accuracy:', accuracy)

In [ ]:
# Unfit the model.
vae.set_weights(initial_weights)

# Train the VAE on KYOTO-2006.
vae.fit(X_train_KYOTO, y_train_KYOTO,
        epochs=50,
        batch_size=2250,
        validation_data=(X_test_KYOTO, y_test_KOYTO))

In [ ]:
# Print the accuracy on the test data.
loss, accuracy = vae.evaluate(X_test_UNSW, y_test_UNSW)
print('Test accuracy:', accuracy)